In [2]:
import sys
sys.path.append("../../Share")
sys.path.append("../")

import config, Processing_same_with_MATLAB, Feature_info
import numpy as np
import pandas as pd

# Feature_info.feature_names
fs = round(10e6 / 2048)  # 4883 Hz
lower_cutoff, upper_cutoff = 100, 600

filter_b, filter_a = Processing_same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

import Train_and_test_one_feature_with_norm_param

Trainer_feature_wise = Train_and_test_one_feature_with_norm_param.main(filter_a,
                                                          filter_b,
                                                          data_files = config.dataset_sub_C,
                                                          default_path = config.default_path_sub_C,
                                                          SUBJECT = "Carlson",
                                                          K = 1) #K : K 이전 데이터 학습, K이후 데이터 테스트

In [ ]:
Normalization_TF = True
feature_idx = 2  # WL

mean_list = [1, 1.5, 2, 2.5, 3, 3.5]
std_list = [0.4, 0.6, 0.8, 1, 1.2]
All_acc, All_param = [], []

for M in mean_list:
    for S in std_list:
        test_acc, X_train_stacked, y_train_stacked, X_test_stacked, y_test_stacked = Trainer_feature_wise.Train_and_test(
            Normalization_TF, M, S, num_feature_set=14, target_feat_idx=feature_idx
        )
        All_acc.append(np.mean(test_acc))
        All_param.append([M, S])

# Find best param
best_idx = np.argmax(All_acc)
best_param = All_param[best_idx]
best_acc = All_acc[best_idx]

In [ ]:
print(f"Best Accuracy: {best_acc:.4f} with mean={best_param[0]}, std={best_param[1]}")

df = pd.DataFrame(All_param, columns=["Mean", "Std"])
df["Accuracy"] = All_acc
print(df.pivot(index="Mean", columns="Std", values="Accuracy"))
df.to_csv(f'Accuracy using Norm params - feature {Feature_info.feature_names[feature_idx]}')